In [1]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot

In [2]:
import pandas as pd
import numpy as np
import os.path as osp

In [3]:
config_file = '/home/semyon/projects/mmdetection/configs/vinbigdata/config.py'
# download the checkpoint from model zoo and put it in `checkpoints/`
# url: http://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
checkpoint_file = '/home/semyon/projects/mmdetection/work_dirs/config_2x/epoch_24.pth'
device = 'cuda:0'
# init a detector
model = init_detector(config_file, checkpoint_file, device=device)

In [4]:
dir_path = '/home/semyon/data/VinBigData/train'
df = pd.read_csv('/home/semyon/data/VinBigData/custom_dfs/weighted_boxes_fusion_iou-0.20_fold-0.csv')

In [4]:
dir_path = '/home/semyon/data/VinBigData/test'
df = pd.read_csv('/home/semyon/data/VinBigData/sample_submission.csv')

In [16]:
new_df = pd.DataFrame(columns=['image_id', 'class_name', 'class_id', 'rad_id', 'x_min', 'y_min',
       'x_max', 'y_max'])
new_df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max


In [5]:
from tqdm.notebook import tqdm

In [6]:
failed = []

In [7]:
i=0
total_image_id = []
total_class_names = []
total_rad_id = []
total_x_min = []
total_y_min = []
total_x_max = []
total_y_max = []
for image_id in tqdm(df.image_id):
    filename = f'{image_id}.npy'
    path = osp.join(dir_path, filename)
    
    try:
        img = np.load(path)
    except:
        failed.append(image_id)
        print(i,image_id)
        continue
    img = np.repeat(img[...,None],3,-1)
    result = inference_detector(model, img)

    for class_name, per_class_result in zip(model.CLASSES, result):
        bboxes = per_class_result[:, :-1]
        scores = per_class_result[:, -1]

        bboxes = np.around(bboxes).astype(np.uint64)
        for score, bbox in zip(scores,bboxes):
            total_image_id.append(image_id)
            total_class_names.append(class_name)
            total_rad_id.append(score)
            total_x_min.append(bbox[0])
            total_y_min.append(bbox[1])
            total_x_max.append(bbox[2])
            total_y_max.append(bbox[3])
#             new_df = new_df.append(dict(image_id=image_id, class_name=class_name, class_id=-1, rad_id=score,
#                    x_min=bbox[0], y_min=bbox[1], x_max=bbox[2], y_max=bbox[3]), ignore_index=True)
    i+=1

  0%|          | 0/4251 [00:00<?, ?it/s]

/home/semyon/projects/mmdetection/mmdet/datasets/utils.py:60: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)


KeyboardInterrupt: 

In [18]:
with open('/home/semyon/projects/vinbigdata/eda/failed_test_files.txt','w') as fp:
    fp.write(str(failed))

In [9]:
len(failed)

0

In [23]:
for image_id in failed:
    new_df = new_df.append(dict(image_id=image_id, class_name='ERROR', class_id=-1, rad_id=0,
                   x_min=0, y_min=0, x_max=1, y_max=1), ignore_index=True)

In [10]:
new_df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,002a34c58c5b758217ed1f584ccbcfe9,Aortic enlargement,-1,0.839313,1160,661,1424,917
1,002a34c58c5b758217ed1f584ccbcfe9,Cardiomegaly,-1,0.950139,832,1207,1911,1605
2,002a34c58c5b758217ed1f584ccbcfe9,ILD,-1,0.121392,1694,1039,2085,1580
3,002a34c58c5b758217ed1f584ccbcfe9,ILD,-1,0.071149,1455,807,2090,1594
4,002a34c58c5b758217ed1f584ccbcfe9,Nodule/Mass,-1,0.164544,1887,1405,1970,1487
...,...,...,...,...,...,...,...,...
69586,ffccf1709d0081d122a1d1f9edbefdf1,Pulmonary fibrosis,-1,0.068183,511,1458,649,1575
69587,ffccf1709d0081d122a1d1f9edbefdf1,Pulmonary fibrosis,-1,0.066415,433,1326,873,1638
69588,ffccf1709d0081d122a1d1f9edbefdf1,Pulmonary fibrosis,-1,0.065110,388,1353,557,1531
69589,ffccf1709d0081d122a1d1f9edbefdf1,Pulmonary fibrosis,-1,0.059114,646,779,826,1083


In [11]:
new_df.image_id.unique().shape

(3000,)

In [11]:
new_df = new_df[new_df.rad_id >= .3]
new_df

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max
0,002a34c58c5b758217ed1f584ccbcfe9,Aortic enlargement,-1,0.839313,1160,661,1424,917
1,002a34c58c5b758217ed1f584ccbcfe9,Cardiomegaly,-1,0.950139,832,1207,1911,1605
7,002a34c58c5b758217ed1f584ccbcfe9,Pleural thickening,-1,0.573166,1425,293,1702,384
22,004f33259ee4aef671c2b95d54e4be68,Aortic enlargement,-1,0.977187,1267,586,1540,911
25,004f33259ee4aef671c2b95d54e4be68,Cardiomegaly,-1,0.481210,1023,1464,1867,1788
...,...,...,...,...,...,...,...,...
69562,ffccf1709d0081d122a1d1f9edbefdf1,Pleural thickening,-1,0.345368,1777,639,2097,871
69575,ffccf1709d0081d122a1d1f9edbefdf1,Pulmonary fibrosis,-1,0.818390,545,779,1010,1294
69576,ffccf1709d0081d122a1d1f9edbefdf1,Pulmonary fibrosis,-1,0.705313,1767,912,2334,1667
69577,ffccf1709d0081d122a1d1f9edbefdf1,Pulmonary fibrosis,-1,0.343321,471,705,1070,1626


In [11]:
new_df.to_csv('/home/semyon/projects/vinbigdata/eda/dfs/result_faster_rcnn_24epoch.csv', index=False)

### Create meta for test

In [28]:
import os.path as osp

In [39]:
meta_df = pd.DataFrame(columns=['image_id', 'dim0', 'dim1'])
meta_df

,image_id,dim0,dim1


In [40]:
meta_df

,image_id,dim0,dim1


In [41]:
dir_name = '/home/semyon/data/VinBigData/test'
for image_id in tqdm(new_df.image_id.unique()):
    path = osp.join(dir_name, f'{image_id}.npy') 
    img = np.load(path)
    h,w = img.shape
#     print(image_id, h, w)
    meta_df = meta_df.append(dict(image_id=image_id, dim0=h, dim1=w), ignore_index=True)

  0%|          | 0/3000 [00:00<?, ?it/s]

In [43]:
meta_df.to_csv('/home/semyon/data/vinbigdata/archive/test_meta.csv',index=False)

In [46]:
osp.split(dir_name)[1]

'test'